In [6]:
import numpy as np # linear algebra
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
#nltk.download('stopwords')
%matplotlib inline

In [7]:
dataset = pd.read_csv("filtered_dataset.csv")
dataset.shape

(50709, 5)

In [8]:
#nltk.download('stopwords')
stemmer = PorterStemmer()
words = stopwords.words("english")
dataset['cleaned'] = dataset['review'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [9]:
dataset.cleaned.head()

0    my son halfway fourth week intuniv we becam co...
1    suboxon complet turn life around i feel health...
2    abilifi chang life there hope i zoloft clonidi...
3    i taken anti depress year improv mostli moder ...
4    i everi medicin sun seem manag hypomania mania...
Name: cleaned, dtype: object

In [20]:
vectorizer = TfidfVectorizer(min_df=3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 5))
final_features = vectorizer.fit_transform(dataset['cleaned']).toarray()
final_features.shape
print("Features are done")

Features are done


In [22]:
from sklearn.linear_model import LogisticRegression
X = dataset['cleaned']
Y = dataset['ratingLabel']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=3000)),
                     ('clf', LogisticRegression(random_state=0))])

model = pipeline.fit(X_train, y_train)
with open('LogisticRegression.pickle', 'wb') as f:
    pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))

/Users/aliakkaya/PycharmProjects/drugRepositioning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

          -1       0.74      0.52      0.61      1994
           0       0.25      0.00      0.00       826
           1       0.81      0.97      0.89      7322

    accuracy                           0.80     10142
   macro avg       0.60      0.50      0.50     10142
weighted avg       0.75      0.80      0.76     10142

[[1030    3  961]
 [ 158    1  667]
 [ 213    0 7109]]
